## Предварительная обработка текста

In [2]:
import re

In [23]:
# Регулярные выражения для поиска нужных фрагментов текста, таких как пунктуация, 
# номера телефонов, адреса электронной почты и URL в разных форматах.
# Регулярные выражения разбиты на логические части и присвоены переменным, 
# которые позволяют их использовать в разных комбинациях,
# в том числе для составления сложных паттернов и объектов для выполнения с ними операций далее
# (методов модуля re)

In [21]:
FRAG_PUNCT_OUTER = r'\[\]*()<>/?!+&€₤$%#“”":;`*=|\\'
FRAG_PUNCT_INNER = r'.,'
RE_PUNCT = re.compile(r'([{punct_outer}])'.format(punct_outer=FRAG_PUNCT_OUTER))
RE_PUNCT_INNER = re.compile(r'([{punct_inner}])'.format(punct_inner=FRAG_PUNCT_INNER))
RE_PUNCT_ALL = re.compile(r'([{punct_inner}{punct_outer}])'.format(punct_inner=FRAG_PUNCT_INNER, punct_outer=FRAG_PUNCT_OUTER))
RE_NUMBERS = re.compile(r'\b[0-9]+(?:(?:\.[0-9]+)+|(?:,[0-9]+))?\b') # Ловит цифры, числа, номера версий.
RE_PHONE = re.compile(r'\+(?:\s*(?:-\s*)?\001){3,}') # Номера телефонов в международном формате с дефисом и без.
FRAG_URL_PATH_COMPONENT = r'[a-zA-Z0-9_.-]+'
FRAG_DOMAIN_NAME = r'(?:[a-zA-Z][a-zA-Z0-9-]*\.)+[a-z]{2,}'
FRAG_URL_FREEFORM = r'[/a-zA-Z0-9_.-]+'
FRAG_URL_QUERYPARAM = r'[a-zA-Z0-9_]+={}'.format(FRAG_URL_FREEFORM)
FRAG_NON_TEXT = r'(?:\001|\002|[{punct_inner}{punct_outer}]+)'.format(punct_inner=FRAG_PUNCT_INNER, punct_outer=FRAG_PUNCT_OUTER)
RE_URL_OR_EMAIL = re.compile(r'\b(?:(http)s?://{dn}(?:/(?:(?:{pathcomp}/)*{pathcomp})?(?:\?{queryparam}(?:\&{queryparam})*)?(?:#{freeform})?)?|(www)\.{dn}\b|{dn}@{dn}\b)'.format(
    dn=FRAG_DOMAIN_NAME, pathcomp=FRAG_URL_PATH_COMPONENT, queryparam=FRAG_URL_QUERYPARAM, freeform=FRAG_URL_FREEFORM))
RE_LEAD_PUNCT = re.compile(r'^\s*(?:{non_text}\s+)*'.format(non_text=FRAG_NON_TEXT))
RE_TRAIL_PUNCT = re.compile(r'(?:\s+{non_text})*\s*$'.format(non_text=FRAG_NON_TEXT))
RE_ALL_PUNCT = re.compile(r'^\s*{non_text}\s*$'.format(non_text=FRAG_NON_TEXT))

In [27]:
# Функция предварительной обработки текста:
# Переводит в нижний регистр, отбивает пунктуацию пробелами, заменяет номера телефонов и числа на заглушки,
# отдельно обрабатывая дробные числа и номера версий, удаляет пунктуацию и числа в начале и конце строки.
# В качестве заглушек выбраны первые пять ASCII символов (крайне маловероятно, что они где-то встретятся в памяти).

In [13]:
def mangle_text_inner(s):
    s = s.strip().lower()
    
    # Отбивка пунктуации 
    s = RE_PUNCT.sub(r' \1 ', s)
    
    # Замена чисел, включая номера версий, на заглушку
    s = RE_NUMBERS.sub(' \001 ', s)

    # Отбивка пунктуации слева от числовой замены
    s = RE_PUNCT_INNER.sub(r' \1 ', s)

    # Постобработка: замена номеров телефона на заглушку
    s = RE_PHONE.sub(' \002 ', s)

    # Постобработка: удаление чисел и пунктуации в начале и конце строки
    s = RE_LEAD_PUNCT.sub('', RE_TRAIL_PUNCT.sub('', RE_ALL_PUNCT.sub('', s)))

    return s

In [ ]:
# Основная функция обработки текста

In [28]:
def mangle_text(s):

    fragments = []

    # Отдельно ищем URL и адреса электронной почты или URL во всех возможных форматах
    while True:
        m = RE_URL_OR_EMAIL.search(s)

        if m is None:
            break

        # Когда находим, обрабатываем текст до и после как фрагменты, после чего соединяем обратно в строку.
        text_before = s[:m.start()]
        if text_before.strip():
            fragments.append(mangle_text_inner(text_before))

        # Заменяем на разные заглушки, в зависимости от типа совпадения.
        if m.group(2) == 'www':
            prefix = '\003'
        elif m.group(1) == 'http':
            prefix = '\004'
        else:
            prefix = '\005'
            
        # Присоединяем заглушку ко всему найденному паттерну
        fragments.append(prefix + m.group(0)) 

        # Проверяем строку дальше
        s = s[m.end():]

    # Если строка содержит какие-то символы кроме пробельных, обрабатываем ее и добавляем к фрагментам
    if s.strip():
        fragments.append(mangle_text_inner(s))

    s = ' '.join(fragments)

    # Разбиваем на слова и снова соединяем, чтобы избавиться от лишних пробелов
    words = s.split()
    s = ' '.join(words)

    return s

In [ ]:
### Примеры работы кода

In [30]:
s = '''
Компания www.basler.com была основана в 1988 году, и вот уже 25 лет работает в сфере видеотехники, 
из которых 15 лет занимается разработкой и производством высококлассных цифровых камер.
'''
m = mangle_text(s)
m

'компания \x03www.basler.com была основана в \x01 году , и вот уже \x01 лет работает в сфере видеотехники , из которых \x01 лет занимается разработкой и производством высококлассных цифровых камер'

In [11]:
s = 'An der Strusbek 60-62'
m = mangle_text(s)
m

'an der strusbek \x01 -'

In [19]:
s = '   Please download pylon version 7.1.9 for '
m = mangle_text(s)
m

'please download pylon version \x01 for'

In [20]:
s = '   Please download pylon version 7.1.9  '
m = mangle_text(s)
m

'please download pylon version'

In [ ]:
s = '   8434 call me at +45  '
m = mangle_text(s)
m